# Velocity Distributions - Generate scale-independent fits

2 sigmas and lambda - $10^{13.5-14}$ $M_\odot$

In [ ]:
import h5py
import numpy as np
from matplotlib.pyplot import figure,show
from swiftsimio import load
import pandas as pd
from scipy.stats import norm
import matplotlib.mlab as mlab
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root, minimize
from lmfit.models import LorentzianModel, VoigtModel, GaussianModel,LognormalModel
from scipy.integrate import fixed_quad, quad, dblquad


In [ ]:
path_hydro= "/net/hypernova/data2/FLAMINGO/L1000N1800/HYDRO_FIDUCIAL/SOAP-HBT/halo_properties_0077.hdf5"
with h5py.File(path_hydro, "r") as handle:
    TotalMass= handle["ExclusiveSphere/100kpc/TotalMass"][:]
    StellarMass = handle["ExclusiveSphere/100kpc/StellarMass"][:]
    COMstellarvelocity=handle["ExclusiveSphere/100kpc/StellarCentreOfMassVelocity"][:]
    COMvelocity=handle["ExclusiveSphere/100kpc/CentreOfMassVelocity"][:]
    Trackid=handle["InputHalos/HBTplus/TrackId"][:]
    HOSTFOFID=handle["InputHalos/HBTplus/HostFOFId"][:]
    HaloCatalogueIndex=handle["InputHalos/HaloCatalogueIndex"][:]
    HOSTHALOINDEX=handle["SOAP/HostHaloIndex"][:]
    FOFMass=handle["InputHalos/FOF/Masses"][:]
    NoofBoundParticles=handle["InputHalos/NumberOfBoundParticles"][:]
    NoofDMParticles=handle["ExclusiveSphere/100kpc/NumberOfDarkMatterParticles"][:]
    COM=handle["ExclusiveSphere/100kpc/CentreOfMass"][:]

In [ ]:
df = pd.DataFrame({
    'HOST_FOF' :  HOSTFOFID,
    'HostHaloIndex':HOSTHALOINDEX, # -1 for central halos
    'Catalogue Index': HaloCatalogueIndex,
    'Track ID' : Trackid,
    'mass':TotalMass,
    'FOFMass':FOFMass,
    'COM v- x':COMvelocity[:,0],
    'COM v- y':COMvelocity[:,1],
    'COM v- z':COMvelocity[:,2],
    'COM - x':COM[:,0],
    'COM - y':COM[:,1],
    'COM - z':COM[:,2],
    'Bound Particles No':NoofBoundParticles,
    'DM Particles No': NoofDMParticles
})
df['INDEX_HOST_HALOS']=np.asarray(df.index)

DF_BOUND_NO_FILTERED= df[df['Bound Particles No']>100]
# display(df)
# display(DF_BOUND_NO_FILTERED)

In [ ]:
def final_velocity(df,lower_mass,upper_mass):
    #filtering FOF groups based on mass
    if upper_mass==-1:
        M_bin=df[df['FOFMass']>=lower_mass] 
    elif lower_mass==-1:
        M_bin=df[(df['FOFMass']<upper_mass)]
    else:
        M_bin=df[(df['FOFMass']>=lower_mass)& (df['FOFMass']<upper_mass)] 
    HOSTINDICES=np.asarray(M_bin.index) #indices of hosts
    display(M_bin)
    print(len(HOSTINDICES)) #check number of host halos
    filtereddf=df.loc[df['HostHaloIndex'].isin(HOSTINDICES)].sort_values(by='HostHaloIndex') #filters all subhalos within each FOF group in M_bin
    print(min(filtereddf['HostHaloIndex']))
    print(len(np.unique(np.array(filtereddf['HostHaloIndex']))))
    # finding number of subhalos in each FOF group
    filtereddf['freq'] = filtereddf.groupby('HostHaloIndex')['HostHaloIndex'].transform('count')
    display(filtereddf)
    df2=filtereddf.groupby('HostHaloIndex')['freq'].mean()
    FINALHOSTINDICES=df2.index
    offsets=df2.values.astype(int)

    M_bin=M_bin[M_bin['INDEX_HOST_HALOS'].isin(FINALHOSTINDICES)] #making sure that the groups corresponding to the subhalos are used 
    HOSTHALO_INDICES=np.asarray(M_bin.index)

    #center of mass vel of each group
    HOSTHALOVEL_X=np.asarray(M_bin['COM v- x'])
    HOSTHALOVEL_Y=np.asarray(M_bin['COM v- y'])
    HOSTHALOVEL_Z=np.asarray(M_bin['COM v- z'])
    #positions
    HOSTHALO_X=np.asarray(M_bin['COM - x'])
    HOSTHALO_Y=np.asarray(M_bin['COM - y'])
    HOSTHALO_Z=np.asarray(M_bin['COM - z'])
    #center of mass vel of each subhalo
    SUBHALO_X=np.asarray(filtereddf['COM v- x'])
    SUBHALO_Y=np.asarray(filtereddf['COM v- y'])
    SUBHALO_Z=np.asarray(filtereddf['COM v- z'])
    
    #subhalo positions
    SUBHALO_x=np.asarray(filtereddf['COM - x'])
    SUBHALO_y=np.asarray(filtereddf['COM - y'])
    SUBHALO_z=np.asarray(filtereddf['COM - z'])

    #relative quantities
    subhalo_relvel_x=np.zeros(len(SUBHALO_X))
    subhalo_relvel_y=np.zeros(len(SUBHALO_X))
    subhalo_relvel_z=np.zeros(len(SUBHALO_X))
    
    subhalo_rel_x=np.zeros(len(SUBHALO_X))
    subhalo_rel_y=np.zeros(len(SUBHALO_X))
    subhalo_rel_z=np.zeros(len(SUBHALO_X))
    print(len(HOSTHALO_INDICES), len(HOSTHALOVEL_X)) #check 
    
    count_lower=0
    count_upper=0
    for i in range(len(HOSTHALO_INDICES)):
        CENTRAL_VELX=HOSTHALOVEL_X[i]
        CENTRAL_VELY=HOSTHALOVEL_Y[i]
        CENTRAL_VELZ=HOSTHALOVEL_Z[i]
        CENTRAL_X=HOSTHALO_X[i]
        CENTRAL_Y=HOSTHALO_Y[i]
        CENTRAL_Z=HOSTHALO_Z[i]
        count_upper+=offsets[i]
        # print(count_lower,count_upper) - check if needed
        subhalo_relvel_x[count_lower:count_upper]=SUBHALO_X[count_lower:count_upper]-CENTRAL_VELX
        subhalo_relvel_y[count_lower:count_upper]=SUBHALO_Y[count_lower:count_upper]-CENTRAL_VELY
        subhalo_relvel_z[count_lower:count_upper]=SUBHALO_Z[count_lower:count_upper]-CENTRAL_VELZ
        subhalo_rel_x[count_lower:count_upper]=CENTRAL_X- SUBHALO_x[count_lower:count_upper]
        subhalo_rel_y[count_lower:count_upper]=CENTRAL_Y-SUBHALO_y[count_lower:count_upper]
        subhalo_rel_z[count_lower:count_upper]=CENTRAL_Z-SUBHALO_z[count_lower:count_upper]
        count_lower+=offsets[i]
    
    # print(count_lower,count_upper) - check if needed
    
    final_velx=subhalo_relvel_x
    final_vely=subhalo_relvel_y
    final_velz=subhalo_relvel_z
    final_relx=subhalo_rel_x
    final_rely=subhalo_rel_y
    final_relz=subhalo_rel_z
    total_vel_xyz=np.concatenate((final_velx,final_vely,final_velz), axis=None) 

    return final_velx,final_vely,final_velz,total_vel_xyz,final_relx,final_rely,final_relz


    


### Maximum likelihood estimation

In [ ]:
def gaussian(x,mu,sigma):
    return  np.exp(-0.5 * ((x -mu) / sigma) ** 2)

def gaussian_integral(mu,sigma,x_i,x_f):
    integral,_=quad(lambda x: gaussian(x,mu,sigma), x_i, x_f)
    return integral
    

def gaussian_neg_log_likelihood_binned(params, bin_edges, bin_heights, bin_width):
    mu,sigma=params
    hist_area=np.sum(bin_heights) 
    fit_integral =(sigma * np.sqrt(2 * np.pi))
    A=hist_area/ fit_integral
    neg_log_L=0
    for i in range(1,len(bin_edges)):
        f_b= A* gaussian_integral(mu,sigma,bin_edges[i-1],bin_edges[i])
        #penalize negative values and zero
        if f_b<=0:
            return 10**11
        n_b=bin_heights[i-1] * bin_width
        neg_log_L+= f_b - (n_b * np.log(f_b))
    return neg_log_L
    

In [ ]:
def laplace(x, mu, b): 
    return (1 / (2 * b)) * np.exp(-np.abs(x - mu) / b)

def laplace_integral(mu,b,x_i,x_f):
    integral,_=quad(lambda x: laplace(x, mu, b), x_i, x_f)
    return integral

def laplace_neg_log_likelihood_binned(params, bin_edges, bin_heights, bin_width):
    mu,b=params
    hist_area=np.sum(bin_heights) 
    fit_integral =laplace_integral(mu,b,-np.inf,np.inf)
    A=hist_area/ fit_integral
    neg_log_L=0
    for i in range(1,len(bin_edges)):
        f_b= A* laplace_integral(mu,b,bin_edges[i-1],bin_edges[i])
        if f_b<0:
            return 10**11
        n_b=bin_heights[i-1] * bin_width
        neg_log_L+= f_b - (n_b * np.log(f_b))
    return neg_log_L

In [ ]:
def lorentzian(x,mu,gamma):
    return 1/ ((1+((x-mu)/gamma)**2))


def lorentzian_integral(mu,gamma,x_i,x_f):
    integral,_=quad(lambda x: lorentzian(x,mu,gamma), x_i, x_f)
    return integral


def lorentzian_neg_log_likelihood_binned(params, bin_edges, bin_heights, bin_width):
    mu,gamma=params
    hist_area=np.sum(bin_heights)
    fit_integral=np.pi*gamma
    A=hist_area/ fit_integral
    neg_log_L=0
    for i in range(1,len(bin_edges)):
        f_b= A*lorentzian_integral(mu,gamma,bin_edges[i-1],bin_edges[i])
        if f_b<0:
            return 10**11
        n_b=bin_heights[i-1] 
        neg_log_L+= f_b - (n_b * np.log(f_b))
    return neg_log_L



### Sum of two gaussians (best model in the thesis):-

In [ ]:
def mod_gaussian(x,sigma,sigma1,lambda_):
    return ((1-lambda_)* (np.exp(-(x)**2 / (2 * sigma**2)) /sigma ) +lambda_*np.exp(-((x)**2/(2*sigma1**2)))/sigma1)/ (np.sqrt(2*np.pi))

def mod_gaussian_integral(sigma,lambda_,lambda2,x_i,x_f):
    integral,_=quad(lambda x: mod_gaussian(x,sigma,lambda_,lambda2), x_i, x_f)
    return integral
    

def mod_gaussian_neg_log_likelihood_binned(params, bin_edges, bin_heights, bin_width):
    sigma,lambda_,lambda2=params
    hist_area=np.sum(bin_heights) 
    fit_integral =(sigma * np.sqrt(2 * np.pi)) *(3*lambda_ + 1 + 105*lambda2)
    A=hist_area/ fit_integral
    neg_log_L=0
    for i in range(1,len(bin_edges)):
        f_b= A* mod_gaussian_integral(sigma,lambda_,lambda2,bin_edges[i-1],bin_edges[i])
        #penalize negative values and zero
        if f_b<=0:
            return 10**11
        n_b=bin_heights[i-1] * bin_width
        neg_log_L+= f_b - (n_b * np.log(f_b))
    return neg_log_L
    

In [ ]:
def plot_distribution_gaussian_mod(f,integral_f,params,data,bins,distname):
    
    bin_heights, bin_edges = np.histogram(data, bins=bins, density=False)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_width= bin_edges[1] - bin_edges[0] 
    bin_widths = np.diff(bin_edges)  # The width of each bin
    number_density = bin_heights / bin_widths  # Normalize by bin width
    # Plot the histogram
    fig=figure(figsize=(7,7))
    frame=fig.add_subplot(1,1,1)
    frame.set_xlabel('Velocity difference v', fontsize=13)
    frame.set_ylabel('Number of galaxies per v', fontsize=13)
    frame.bar(bin_centers, number_density, width=bin_width, align='center')
    DAT=np.linspace(np.min(data),np.max(data),1000)
    hist_area=np.sum(bin_heights)
    fit_integral = integral_f(*params,-np.inf,np.inf) 
    A=hist_area/ fit_integral
    frame.plot(DAT,A*f(DAT,*params),'-', label=distname,color='red')
    # frame.set_yscale("log")
    frame.legend()

## trial stuff

In [ ]:
M15_final_velx,M15_final_vely,M15_final_velz,M15_total_Vel,M15_final_x,M15_final_y,M15_final_z=final_velocity(DF_BOUND_NO_FILTERED,10**3.5,10**4)

In [ ]:
df_scale=pd.DataFrame({
    'Relative pos - x':M15_final_x,
    'Relative pos - y':M15_final_y,
    'Relative pos - z':M15_final_z,
    'Relative radial distance':np.sqrt(M15_final_x**2+M15_final_y**2+M15_final_z**2),
    'Relative velocity - x':M15_final_velx,
    'Relative velocity - y':M15_final_vely,
    'Relative velocity - z':M15_final_velz,
})

In [ ]:
df2=df_scale.sort_values(by=['Relative radial distance'])

len(df2['Relative velocity - z']) + len(df2['Relative velocity - x']) + len(df2['Relative velocity - y'])
print(np.array(df2['Relative radial distance']))

## Trial - binned 


In [ ]:
r_values = df2['Relative radial distance']  
x_values =df2['Relative velocity - x']
y_values=df2['Relative velocity - y']
z_values= df2['Relative velocity - z']
r_bins=[(0,0.16),(0.16,0.225),(0.23,0.29),(0.29,0.325),(0.325,0.3625),(0.3625,0.4),(0.4,0.45),(0.45,0.5),(0.5,0.55),(0.55,0.6),(0.6,0.65),(0.65,0.7),(0.7,0.8),(0.8,1.0),(1.0,1.2),(1.2,5)] # for now
unbinned_velocity_values_in_each_rbin=[]
def bin_x_values(r_values, x_values, y_values,z_values, r_bins):
    binned_data = []
    c=0
    for r_min, r_max in r_bins:
        mask_r = (r_values >= r_min) & (r_values < r_max)
        x_vals = x_values[mask_r] 
        y_vals= y_values[mask_r]
        z_vals= z_values[mask_r]
        unbinned_velocity_values_in_each_rbin.append(np.concatenate((x_vals,y_vals,z_vals), axis=None))
        counts, bins = np.histogram(np.concatenate((x_vals,y_vals,z_vals), axis=None) , bins=80) 
        x_bins_list = [(float(bins[i]), float(bins[i+1]), int(counts[i])) for i in range(len(counts))]
        binned_data.append((float(r_min), float(r_max), x_bins_list))
    return binned_data

binned_data = bin_x_values(r_values, x_values, y_values,z_values, r_bins)


In [ ]:
def f_x(x, sigma, sigma1, lambda_):
    return (((1-lambda_)* (np.exp(-(x)**2 / (2 * sigma**2)) /sigma )) + ((lambda_*np.exp(-((x)**2/(2*sigma1**2))))/sigma1)) / (np.sqrt(2*np.pi))
    
# Scale-dependent parameter functions 
def sigma_r(r, a, b, c,n):
    return a * r**n + b * r + c


def sigma1_r(r, m, b):
    return m * r + b


def lambda_r(r, A, B, C):
    return A * np.exp(-B * r) + C

In [ ]:

# Computing expected count in a single (r, x) bin


def expected_count_single_bin(r_min, r_max, x_min, x_max, params):
    def integrand(x, r):
        sigma = sigma_r(r, *params[:4])  
        sigma1 = sigma1_r(r, *params[4:6]) 
        lam = lambda_r(r, *params[6:9])   
        return f_x(x, sigma, sigma1, lam)  
    #inner - x integral, outer integral- r
    total, _ = dblquad(integrand, r_min, r_max, lambda r: x_min, lambda r: x_max)
    return total


def neg_log_likelihood(params, binned_data):
    logL = 0
    for r_min, r_max, x_bins in binned_data:
        for (x_min, x_max, N_obs) in x_bins:
            P = expected_count_single_bin(r_min, r_max, x_min, x_max, params)
            if P <= 0:
                return 1e10  # Avoid log(0), should not be happening though
            logL += N_obs * np.log(P) - P
    return -logL


# initial_params = [] - provide initial guesses for each parameter!
# Minimization
result = minimize(neg_log_likelihood, initial_params, args=(binned_data,),bounds=[..]) #Provide bounds for each parameter!


# print(result)

# Plots 

In [ ]:

params = result.x
print("Optimized parameters:", params)

In [ ]:
r_max=5 # Mpc - provide upper limit of radial separation

#Marginalize dist with uniform P(r) = 1/r_max
def marginalized_P_x(x, params, r_max):
    integral_r, _ = quad(lambda r: f_x(x, sigma_r(r, *params[:4]),sigma1_r(r, *params[4:6]), lambda_r(r, *params[6:9])),
                         0, r_max)
    return integral_r / r_max  


x_range = np.linspace(-3000, 3000, 6000)
marginalized_dist = np.array([marginalized_P_x(x, params, r_max) for x in x_range])

total_count = sum(N_obs for _, _, x_bins in binned_data for (_, _, N_obs) in x_bins)
print("Total observed count:", total_count)


def normalization_const_r(params, r_max):
    #f(y,x) form, with y being the inner integral variable!!
    def integrand(r, x, params):
        sigma = sigma_r(r, *params[:4])    
        sigma1 = sigma1_r(r, *params[4:6]) 
        lam = lambda_r(r, *params[6:9]) 
        return f_x(x, sigma, sigma1, lam)  

    # Integrate over r first then x
    total_area, _ = dblquad(integrand, -np.inf, np.inf, lambda x: 0, lambda x: r_max, args=(params,))
    return total_area / (r_max)
    
A = normalization_const_r(params, r_max)
print("Normalization constant:", A)

scaled_p = marginalized_dist * total_count / (A )
plt.plot(x_range, color="red", linewidth=2)
plt.title("Velocity distribution")
plt.show()


bin_heights, bin_edges = np.histogram(np.concatenate((x_values,y_values,z_values),axis=None),bins=80, density=False)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
bin_width= bin_edges[1] - bin_edges[0] 
number_density = bin_heights / bin_width
plt.bar(bin_centers, number_density, width=bin_width, align='center')
plt.plot(x_range, scaled_p, color="red", linewidth=2,label=f"N={total_count:.0f}")
plt.xlabel("Velocity difference v", fontsize=16)
plt.ylabel("Number of galaxies per v", fontsize=16)
plt.tick_params(axis='both', which='major', length=6, width=2, labelsize=14)
# plt.savefig("Marginalized_distribution_allr_M13_5_14.pdf", dpi=500)
plt.show()



In [ ]:
def marginalized_P_x_specific_r(x, params, r_min , r_max):
    integral_r, _ = quad(lambda r: f_x(x, sigma_r(r, *params[:4]),sigma1_r(r, *params[4:6]), lambda_r(r, *params[6:9])),
                         r_min, r_max)
    return integral_r / ((r_max - r_min))




def normalization_const_specific_r(params, r_min, r_max):
    #f(y,x) form with y being the inner integral variable!!
    def integrand(r, x, params):
        sigma = sigma_r(r, *params[:4])    
        sigma1 = sigma1_r(r, *params[4:6]) 
        lam = lambda_r(r, *params[6:9]) 
        return f_x(x, sigma, sigma1, lam) 

    # Integrate over r first then x
    total_area, _ = dblquad(integrand, -np.inf, np.inf, lambda x: r_min, lambda x: r_max, args=(params,))
    return total_area / (r_max - r_min)


def marginalized_P_plots(velocity_values,r_bins,opt_params):
    """
    velocity_values = velocity values inside each r bin (unbinned)
    r_bins = list with bin edges of each r bin
    opt_params = fitted parameters of scale dependent functions
    """
    # fig=figure(figsize=(10,10))
    for i in range(len(r_bins)):
        #Calculate normalization constant
        fig=figure(figsize=(7,5))
        frame=fig.add_subplot(1,1,1)
        A = normalization_const_specific_r(opt_params, r_bins[i][0],r_bins[i][1])
        # print(f"Normalization constant for bin {r_bins[i][0]} Mpc - {r_bins[i][1]} Mpc = {A} ")
        
        #Generate bin-marginalized distribution inside each r bin
        x_range = np.linspace(np.min(velocity_values[i]), np.max(velocity_values[i]), 7000)

        marginalized_dist_specific_r = np.array([marginalized_P_x_specific_r(x, opt_params, r_bins[i][0],r_bins[i][1]) for x in x_range])

        
        # frame=fig.add_subplot(len(r_bins),1,i+1 )
        bin_heights, bin_edges = np.histogram(velocity_values[i], bins=30, density=False)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bin_width= bin_edges[1] - bin_edges[0] 
        number_density = bin_heights / bin_width
        hist_area=len(velocity_values[i])
        scaled_p = marginalized_dist_specific_r * hist_area / A
        frame.bar(bin_centers, number_density, width=bin_width, align='center')
        frame.plot(x_range, scaled_p, color="red", linewidth=2, label=f"N={hist_area:.0f}")
        frame.set_xlabel('Velocity difference v', fontsize=13)
        frame.set_ylabel('Number of galaxies per v', fontsize=13)
        frame.tick_params(axis='both', which='major', length=6, width=2, labelsize=14)
        frame.legend(fontsize=12.5, loc="upper right")
        fig.tight_layout()
        #TO SAVE IMAGE
        # fig.savefig(f"Marginalized_distribution_allr_M13_5_14__bin_{r_bins[i][0]:.1f}_{r_bins[i][1]:.1f}.pdf", dpi=500)
        show()

        
marginalized_P_plots(unbinned_velocity_values_in_each_rbin,r_bins,params,10**14,10**14.5)   

Comments from Sowmya:

I could have used MCMC fitting procedures instead of the one I used here, so I would suggest adding that instead of what I did. Additionally, I should have explored more combinations of scale-dependent functions for the parameters. Good luck, and I would really appreciate it if you could keep me updated on the progress made in finding/improving the model at the end of your project!